In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import os
from multiprocessing import Pool

In [ ]:
#preprocessing on a single image

plt.figure(figsize=(20,20))
img_0_orig = cv2.imread(img_0_path)
img_0_orig = cv2.cvtColor(img_0_orig, cv2.COLOR_BGR2RGB)
img_0 = cv2.imread(img_0_path)
img_0 = cv2.cvtColor(img_0, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(img_0,(3,3),0)
sharp_mask = np.subtract(img_0, blur)
img_0 = cv2.addWeighted(img_0,1, sharp_mask,10, 0)
ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel_1 = np.ones((5,5),np.uint8)
kernel_2 = np.ones((1,1),np.uint8)
opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel_1)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_2)
mask = cv2.bitwise_not(closing)
mask = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)
img = cv2.add(img_0_orig,mask)
blur_1 = cv2.GaussianBlur(img, (13,13), 0)
sharp_mask_1 = np.subtract(img,blur_1)
sharp_mask_1 = cv2.GaussianBlur(sharp_mask_1, (7,7), 0)
img = cv2.addWeighted(img,1,sharp_mask_1,-10, 0)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img, cmap = 'gray')
print(img_0_orig.shape)

In [2]:
def processing_image(image_file, in_folder:Path, out_folder:Path):
    image_path = str(in_folder / image_file)
    img_0_orig = cv2.imread(image_path)
    img_0_orig = cv2.cvtColor(img_0_orig, cv2.COLOR_BGR2RGB)
    img_0 = cv2.imread(image_path)
    img_0 = cv2.cvtColor(img_0, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(img_0,(3,3),0)
    sharp_mask = np.subtract(img_0, blur)
    img_0 = cv2.addWeighted(img_0,1, sharp_mask,10, 0)
    ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel_1 = np.ones((5,5),np.uint8)
    kernel_2 = np.ones((1,1),np.uint8)
    opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel_1)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_2)
    mask = cv2.bitwise_not(closing)
    mask = cv2.cvtColor(closing, cv2.COLOR_GRAY2RGB)
    img = cv2.add(img_0_orig,mask)
    blur_1 = cv2.GaussianBlur(img, (13,13), 0)
    sharp_mask_1 = np.subtract(img,blur_1)
    sharp_mask_1 = cv2.GaussianBlur(sharp_mask_1, (7,7), 0)
    img = cv2.addWeighted(img,1,sharp_mask_1,-10, 0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    cv2.imwrite(str(out_folder/image_file), img)

In [ ]:
train_path = "../input/train_images/"
test_path = "../input/test_images/"

train_output = "../input/processed_train/"
test_output = "../input/processed_test/"

if not os.path.isdir(train_output):
    os.mkdir(train_output)

if not os.path.isdir(test_output):
    os.mkdir(test_output)

In [ ]:
files = os.listdir(train_path)
train_images = []
for file in files:
    if not os.path.isdir(file):
        train_images.append(file)

files = os.listdir(test_path)
test_images = []
for file in files:
    if not os.path.isdir(file):
        test_images.append(file)

In [ ]:
print(len(train_images))
print(len(test_images))

# print(train_images[0])
# processing_image(train_images[0], Path(train_path), Path(train_output))

In [ ]:
%%time

pool = Pool(processes=8)
for image_file in train_images:
    pool.apply(processing_image, (image_file, Path(train_path), Path(train_output), ))
pool.close()
pool.join()

In [ ]:
%%time

pool = Pool(processes=8)
for image_file in test_images:
    pool.apply(processing_image, (image_file, Path(test_path), Path(test_output), ))
pool.close()
pool.join()